In [1]:
import abc
import math
import time

import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import IPython

In [2]:
def to_polar(x, y):
    r = torch.sqrt(x**2 + y**2)
    theta = torch.atan2(y, x)
    return r, theta

def trapquad(f, a, b, N, *args):
    x = torch.linspace(a, b, N+1) #(n,)
    dx = (b - a)/N
    y = f(x, *args) #(n, *argdims)
    value = 0.5*dx*torch.sum((y[1:, ...] + y[:-1, ...]), axis=0) #(*argdims)
    return value

def circle_phi(r, N=100):
    def integrand(theta, r):
        if not isinstance(r, float):
            theta = theta.view(*(theta.shape + (1,)*theta.ndim))
        return 1/torch.sqrt(1 + r**2 - 2*r*torch.cos(theta))
    integral = trapquad(integrand, 0, 2*math.pi, N, r)
    integral = torch.nan_to_num(integral, 0.0)
    return integral

def upper_mask(N):
    return torch.triu(torch.ones(N, N) * float('inf'))

def diagonal_mask(N):
    return torch.diag(torch.ones(N) * float('inf'))

In [31]:
class FieldObject(object):
    def potential(x, y, charge, coupling):
        pass

class Ring(FieldObject):
    def __init__(self, radius, charge_density=1.0, x0=0.0, y0=0.0):
        super().__init__()
        self.radius = radius
        self.charge_density = charge_density
        self.x0 = x0
        self.y0 = y0
        
    def potential(self, x, y, charge, coupling=1.0):
        r, _ = to_polar(x - self.x0, y - self.y0)
        value = coupling*self.charge_density*charge*circle_phi(r/self.radius)
        return value


class HorizontalLine(FieldObject):
    def __init__(self, y0, charge_density=1.0):
        self.charge_density = charge_density
        self.y0 = y0
        
    def potential(self, x, y, charge, coupling=1.0):
        value = -coupling*self.charge_density*charge*torch.log(torch.abs(y - self.y0))
        return value


class VerticalLine(FieldObject):
    def __init__(self, x0, charge_density=1.0):
        self.charge_density = charge_density
        self.x0 = x0
        
    def potential(self, x, y, charge, coupling=1.0):
        value = -coupling*self.charge_density*charge*torch.log(torch.abs(x - self.x0))
        return value
    

class Hash(FieldObject):
    def __init__(self, l, charge_density=1.0, x0=0.0, y0=0.0):
        self.l = l
        self.charge_density = charge_density
        self.x0 = x0
        self.y0 = y0
        self._set_lines()
    
    def potential(self, x, y, charge, coupling=1.0):
        value = self._upper.potential(x, y, charge, coupling) + \
                self._lower.potential(x, y, charge, coupling) + \
                self._left.potential(x, y, charge, coupling) + \
                self._right.potential(x, y, charge, coupling)
        return value
    
    def _set_lines(self):
        self._upper = HorizontalLine(self.y0 + self.l/2, self.charge_density)
        self._lower = HorizontalLine(self.y0 - self.l/2, self.charge_density)
        self._left = VerticalLine(self.x0 + self.l/2, self.charge_density)
        self._right = VerticalLine(self.x0 - self.l/2, self.charge_density)
        

In [32]:
class PointSystem(torch.nn.Module):
    def __init__(self, x, y, px=None, py=None, mass=1.0, charge=1.0):
        super().__init__()
        x = x #(n, )
        y = y #(n, )
        px = torch.zeros_like(x) if px is None else px
        py = torch.zeros_like(y) if py is None else py
        self._register_positions_as_parameters(x, y, px, py)
        self._assert_positions()
        #Assert blablabla
        self.charge = charge #(n, )
        self.mass = mass #(n, )
        
    def kinetic_energy(self):
        energy = 1/(2*self.mass)*torch.sum((self.px**2 + self.py**2))
        return energy
    
    def internal_energy(self, coupling=1.0):
        dists = torch.cdist(system.xy, system.xy) + diagonal_mask(self.dim)
        energies = coupling*self.charge**2/dists
        energy = torch.sum(energies)
        return energy
        
    def external_energy(self, objects=None, coupling=1.0):
        if objects is None:
            return 0.0
        energies = torch.stack([obj.potential(self.x, self.y, self.charge, coupling) for obj in objects])
        energy = torch.sum(energies)
        return energy
    
    def potential_energy(self, objects=1.0, coupling=1.0):
        return self.internal_energy(coupling) + self.external_energy(objects, coupling)
    
    def hamiltonian(self, objects=None, coupling=1.0):
        ke = self.kinetic_energy() 
        pe = self.potential_energy(objects, coupling)
        return ke + pe
    
    @property
    def xy(self):
        return torch.stack([self.x, self.y], axis=-1)
    
    @property
    def pxy(self):
        return torch.stack([self.px, self.py], axis=-1)
    
    @property
    def vx(self):
        return self.x/self.px
    
    @property
    def vy(self):
        return self.y/self.py
    
    @property
    def vxy(self):
        return torch.stack([self.vx, self.vy], axis=-1)
    
    @property
    def dim(self):
        return self.x.shape[-1]
    
    def _assert_positions(self):
        assert (self.x.ndim, self.y.ndim, self.px.ndim, self.py.ndim) == ((1,)*4)
        assert (self.x.shape[-1], self.y.shape[-1], self.px.shape[-1], self.py.shape[-1]) == ((self.dim,)*4)
        
    def _register_positions_as_parameters(self, x, y, px, py):
        self.x = torch.nn.Parameter(x)
        self.y = torch.nn.Parameter(y)
        self.px = torch.nn.Parameter(px)
        self.py = torch.nn.Parameter(py)
        return

In [33]:
def hamiltonian_rhs(system, objects, coupling):
    hamiltonian = system.hamiltonian(objects, coupling)
    hamiltonian.backward()
    x_rhs = system.px.grad
    y_rhs = system.py.grad
    px_rhs = -system.x.grad
    py_rhs = -system.x.grad
    return x_rhs, y_rhs, px_rhs, py_rhs

def force_rhs(system, objects, coupling):
    potential_energy = system.potential_energy(objects, coupling)
    potential_energy.backward()
    x_rhs = system.px.detach()/system.mass
    y_rhs = system.py.detach()/system.mass
    px_rhs = -system.x.grad
    py_rhs = -system.y.grad
    return x_rhs, y_rhs, px_rhs, py_rhs    

def euler_step(dt, system, objects=None, coupling=1.0):
    system.zero_grad()
    dx, dy, dpx, dpy = hamiltonian_rhs(system, objects, coupling)
    with torch.no_grad():
        system.x += dx*dt
        system.y += dy*dt
        system.px += dpx*dt
        system.py += dpy*dt
        
def leapfrog_step(dt, system, objects=None, coupling=1.0):
    system.zero_grad()
    dx, dy, dpx, dpy = force_rhs(system, objects, coupling)
    with torch.no_grad():
        #system.x += dx*dt
        #system.y += dy*dt
        #system.px += dpx*dt
        #system.py += dpy*dt
        system.px += dpx*dt
        system.py += dpy*dt
        system.x += system.px*dt/system.mass
        system.y += system.py*dt/system.mass

In [34]:
#x' = v = p/m
#p' = -grad U(x)

In [51]:
#R = 2.0

#n = 5
#r = torch.sqrt(torch.rand(n))
#theta = torch.rand(n)*2*math.pi
#x = r*torch.cos(theta)
#y = r*torch.sin(theta)

n = 5
x = torch.rand(n)*2*R - R
y = torch.rand(n)*2*R - R
# d = 0.35
# x = torch.tensor([d, 0.0, -2*d, 0.0])
# y = torch.tensor([0.0, 2*d, 0.0, -d])

In [55]:
system = PointSystem(x, y)
ring = Hash(2*R, 1.0)
objects = [ring]

TypeError: zeros_like(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [56]:
x

array([ 0.29053697,  1.056357  , -1.559193  ,  1.5956596 , -1.3977557 ],
      dtype=float32)

In [54]:
t = 100.0
dt = 0.01
n = int(t/dt) + 1
ring_tplot = np.linspace(0, 2*np.pi, 101)
ring_xplot = R*np.cos(ring_tplot)
ring_yplot = R*np.sin(ring_tplot)
hash_xplot = [-R, R, R, -R, -R]
hash_yplot = [-R, -R, R, R, -R]
plt.plot([], [])
for i in range(n):
    leapfrog_step(dt, system, objects, coupling=-1.0)
    x, y = system.x.detach().numpy(), system.y.detach().numpy()
    print(x**2 )
#     if np.max(x**2 + y**2) > R**2:
#         break
    if max(np.max(np.abs(x)), np.max(np.abs(y))) > R:
        break
    #plt.plot(ring_xplot, ring_yplot, color='red')
    plt.plot(hash_xplot, hash_yplot, color='red')
    scatter = plt.scatter(x, y, color='blue')
    plt.xlim(-2.5, 2.5)
    plt.ylim(-2.5, 2.5)
    plt.title('Time = %f'%(dt*i))
    plt.show()
    time.sleep(1/30)
    IPython.display.clear_output(wait=True)

[0.08441173 1.1158901  2.431083   2.5461297  1.9537212 ]


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [ ]:
x**2 + y**2